# 🚀 Jetson Emotion Detection: Train + TensorRT Inference

In [1]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [2]:

# === CONFIG ===
image_size = (48, 48)
batch_size = 32
epochs = 5
train_dir = "dataset/train"
test_dir = "dataset/test"


In [3]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
class_labels = list(train_generator.class_indices.keys())


Found 28709 images belonging to 7 classes.
Found 7182 images belonging to 8 classes.


In [4]:

model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 256)         295168    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 4, 4, 256)         0

In [5]:

early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stop]
)


Epoch 1/25
898/898 [==============================] - ETA: 0s - loss: 1.8131 - accuracy: 0.2504

InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/aarons/.local/lib/python3.8/site-packages/ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "/home/aarons/.local/lib/python3.8/site-packages/traitlets/config/application.py", line 1075, in launch_instance
      app.start()
    File "/home/aarons/.local/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 739, in start
      self.io_loop.start()
    File "/home/aarons/.local/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 205, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_foreve

In [ ]:

# === SAVE FOR TensorRT ===
os.makedirs("saved_model", exist_ok=True)
model.save("saved_model")
print("✅ Model saved to 'saved_model/'")


In [ ]:

# === CONVERT TO TENSORRT ===
from tensorflow.python.compiler.tensorrt import trt_convert as trt

converter = trt.TrtGraphConverterV2(input_saved_model_dir="saved_model")
converter.convert()
converter.save("trt_model")
print("✅ TensorRT optimized model saved to 'trt_model/'")


In [ ]:

# === RUN TENSORRT INFERENCE ===
import tensorflow as tf

image_dir = "images"
infer_model = tf.saved_model.load("trt_model")
infer_fn = infer_model.signatures['serving_default']

for img_file in os.listdir(image_dir)[:5]:
    path = os.path.join(image_dir, img_file)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, image_size)
    norm = img.astype("float32") / 255.0
    norm = np.expand_dims(norm, axis=(0, -1))  # Shape: (1, 48, 48, 1)
    tensor_input = tf.convert_to_tensor(norm)

    result = infer_fn(tensor_input)
    preds = list(result.values())[0].numpy()[0]
    label = class_labels[np.argmax(preds)]
    conf = np.max(preds)

    print(f"{img_file} → {label} ({conf*100:.2f}%)")
    plt.imshow(cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB))
    plt.title(f"{label} ({conf*100:.2f}%)")
    plt.axis("off")
    plt.show()
